In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import numpy as np
import requests
from config import rapid_api_key
from config import rapidapi_host
from pprint import pprint
import datetime

In [18]:
# os.getcwd()
# os.chdir('C:\\gitlab\\covid19')

In [19]:
#Import files to use with API's
county_path = 'Resources/us_counties_states.csv'
date_path = 'Resources/dates.csv'

In [20]:
us_counties_df = pd.read_csv(county_path)
mask1 = us_counties_df['State'] == 'California'
mask2 = us_counties_df['State'] == 'Indiana'

us_counties_df = us_counties_df[mask1 | mask2]
us_counties_df.head()

,County,State,"County, State"
186,Alameda,California,"Alameda,California"
187,Alpine,California,"Alpine,California"
188,Amador,California,"Amador,California"
189,Butte,California,"Butte,California"
190,Calaveras,California,"Calaveras,California"


In [21]:
date_df = pd.read_csv(date_path)

In [22]:
mask1 = date_df['Date']>'2020-10-10'
date_df=date_df[mask1]
date_df

,Date
203,2020-10-11
204,2020-10-12
205,2020-10-13


In [23]:
county_list = []
state_list = []
county_list = us_counties_df['County, State'].tolist()
date_list = date_df['Date'].tolist()

In [24]:
# Run API to collect county level data
# url = "https://covid-19-statistics.p.rapidapi.com/reports"
# headers = {'x-rapidapi-host': rapidapi_host,'x-rapidapi-key': rapid_api_key}

# # County Lists
# county_state_list = []
# new_county_list = []
# tot_county_cases = []
# new_county_cases = []
# tot_county_deaths = []
# new_county_deaths = []
# county_date = []
# county_lat = []
# county_lng = []

# record_counter = 0
# set_counter = 0

# for date_val in date_list:    
#     try:        
#         set_counter += 1

#         for name in county_list:
#             value = name.split(',')
#             county = value[0]
#             state = value[1]
#             country_state = "US " + state
#             record_counter +=1

#             querystring = {"region_province":state,"iso":"USA","region_name":"US","city_name":county, \
#                            "date":date_val,"q":country_state}   

#             response = requests.request("GET", url, headers=headers, params=querystring).json()    

#             try:   
#                 # Add County information
#                 county_state_list.append(response['data'][0]['region']['province'])
#                 new_county_list.append(response['data'][0]['region']['cities'][0]['name'])
#                 tot_county_cases.append(response['data'][0]['region']['cities'][0]['confirmed'])
#                 new_county_cases.append(response['data'][0]['region']['cities'][0]['confirmed_diff'])
#                 tot_county_deaths.append(response['data'][0]['region']['cities'][0]['deaths'])
#                 new_county_deaths.append(response['data'][0]['region']['cities'][0]['deaths_diff'])
#                 county_date.append(response['data'][0]['region']['cities'][0]['date'])
#                 county_lat.append(response['data'][0]['region']['cities'][0]['lat'])
#                 county_lng.append(response['data'][0]['region']['cities'][0]['long'])

#                 if record_counter > 150:
#                     record_counter = 1
#                     set_counter += 1

#                 print(f'Processing record {record_counter} of set {set_counter} | {county}, {state}, {date_val}.')

#             except:
#                 record_counter -= 1
#                 print(f'Error: County: {county} and State:{state} not found for: {date_val}')
                
#         set_counter += 1
        
#     except:
#         print(f'Date Error: {date_val} was not processed')
            
# print(f'---------------------------------Process Completed----------------------------------------')


Processing record 1 of set 1 | Alameda, California, 2020-10-11.
Processing record 2 of set 1 | Alpine, California, 2020-10-11.
Processing record 3 of set 1 | Amador, California, 2020-10-11.
Processing record 4 of set 1 | Butte, California, 2020-10-11.
Processing record 5 of set 1 | Calaveras, California, 2020-10-11.
Processing record 6 of set 1 | Colusa, California, 2020-10-11.
Processing record 7 of set 1 | Contra Costa, California, 2020-10-11.
Processing record 8 of set 1 | Del Norte, California, 2020-10-11.
Processing record 9 of set 1 | El Dorado, California, 2020-10-11.
Processing record 10 of set 1 | Fresno, California, 2020-10-11.
Processing record 11 of set 1 | Glenn, California, 2020-10-11.
Processing record 12 of set 1 | Humboldt, California, 2020-10-11.
Processing record 13 of set 1 | Imperial, California, 2020-10-11.
Processing record 14 of set 1 | Inyo, California, 2020-10-11.
Processing record 15 of set 1 | Kern, California, 2020-10-11.
Processing record 16 of set 1 | Kin

Processing record 130 of set 1 | Shelby, Indiana, 2020-10-11.
Processing record 131 of set 1 | Spencer, Indiana, 2020-10-11.
Processing record 132 of set 1 | Starke, Indiana, 2020-10-11.
Processing record 133 of set 1 | Steuben, Indiana, 2020-10-11.
Processing record 134 of set 1 | Sullivan, Indiana, 2020-10-11.
Processing record 135 of set 1 | Switzerland, Indiana, 2020-10-11.
Processing record 136 of set 1 | Tippecanoe, Indiana, 2020-10-11.
Processing record 137 of set 1 | Tipton, Indiana, 2020-10-11.
Processing record 138 of set 1 | Union, Indiana, 2020-10-11.
Processing record 139 of set 1 | Vanderburgh, Indiana, 2020-10-11.
Processing record 140 of set 1 | Vermillion, Indiana, 2020-10-11.
Processing record 141 of set 1 | Vigo, Indiana, 2020-10-11.
Processing record 142 of set 1 | Wabash, Indiana, 2020-10-11.
Processing record 143 of set 1 | Warren, Indiana, 2020-10-11.
Processing record 144 of set 1 | Warrick, Indiana, 2020-10-11.
Processing record 145 of set 1 | Washington, India

Processing record 108 of set 4 | Miami, Indiana, 2020-10-12.
Processing record 109 of set 4 | Monroe, Indiana, 2020-10-12.
Processing record 110 of set 4 | Montgomery, Indiana, 2020-10-12.
Processing record 111 of set 4 | Morgan, Indiana, 2020-10-12.
Processing record 112 of set 4 | Newton, Indiana, 2020-10-12.
Processing record 113 of set 4 | Noble, Indiana, 2020-10-12.
Processing record 114 of set 4 | Ohio, Indiana, 2020-10-12.
Processing record 115 of set 4 | Orange, Indiana, 2020-10-12.
Processing record 116 of set 4 | Owen, Indiana, 2020-10-12.
Processing record 117 of set 4 | Parke, Indiana, 2020-10-12.
Processing record 118 of set 4 | Perry, Indiana, 2020-10-12.
Processing record 119 of set 4 | Pike, Indiana, 2020-10-12.
Processing record 120 of set 4 | Porter, Indiana, 2020-10-12.
Processing record 121 of set 4 | Posey, Indiana, 2020-10-12.
Processing record 122 of set 4 | Pulaski, Indiana, 2020-10-12.
Processing record 123 of set 4 | Putnam, Indiana, 2020-10-12.
Processing rec

Processing record 86 of set 7 | Harrison, Indiana, 2020-10-13.
Processing record 87 of set 7 | Hendricks, Indiana, 2020-10-13.
Processing record 88 of set 7 | Henry, Indiana, 2020-10-13.
Processing record 89 of set 7 | Howard, Indiana, 2020-10-13.
Processing record 90 of set 7 | Huntington, Indiana, 2020-10-13.
Processing record 91 of set 7 | Jackson, Indiana, 2020-10-13.
Processing record 92 of set 7 | Jasper, Indiana, 2020-10-13.
Processing record 93 of set 7 | Jay, Indiana, 2020-10-13.
Processing record 94 of set 7 | Jefferson, Indiana, 2020-10-13.
Processing record 95 of set 7 | Jennings, Indiana, 2020-10-13.
Processing record 96 of set 7 | Johnson, Indiana, 2020-10-13.
Processing record 97 of set 7 | Knox, Indiana, 2020-10-13.
Processing record 98 of set 7 | Kosciusko, Indiana, 2020-10-13.
Processing record 99 of set 7 | LaGrange, Indiana, 2020-10-13.
Processing record 100 of set 7 | Lake, Indiana, 2020-10-13.
Processing record 101 of set 7 | LaPorte, Indiana, 2020-10-13.
Processi

In [30]:
# Create dataframe for county level data
# covid_county_new_df = pd.DataFrame({
#     "State" : county_state_list,
#     "County" : new_county_list,
#     "Total Cases" : tot_county_cases,
#     "New Cases" : new_county_cases,
#     "Total Deaths" : tot_county_deaths,
#     "New Deaths" : new_county_deaths,
#     "Date" : county_date,
#     "Latitude" : county_lat,
#     "Longitude" : county_lng   
# })

# covid_county_new_df.to_csv('Resources/covid_county_df.csv', mode='a', index=False, header=False)

In [71]:
covid_county_new_df.head(2)

,State,County,Total Cases,New Cases,Total Deaths,New Deaths,Date,Latitude,Longitude
0,California,Alameda,22149,77,441,0,2020-10-11,37.64629437,-121.8929271
1,California,Alpine,3,0,0,0,2020-10-11,38.59678594,-119.8223594


In [32]:
# Use this API call to get the state information without the county level detail
# us_state_df = pd.read_csv('Resources/us_counties_states.csv')

# state_list = []

# s_list = us_state_df['State'].tolist()

# for item in s_list:
#     if item not in state_list:
#         state_list.append(item)

# # State lists
# new_state = []
# total_cases_state = []
# new_cases_state = []
# tot_deaths_state = []
# new_deaths_state = []
# fatality_rate_state = []
# date_state = []
# lat_state = []
# long_state = []

# url = "https://covid-19-statistics.p.rapidapi.com/reports"
# headers = {'x-rapidapi-host': rapidapi_host,'x-rapidapi-key': rapid_api_key}

# state_record_counter = 0
# state_set_counter = 0

# for date_val in date_list:
    
#     try:    
#         state_set_counter += 1
    
#         for state in state_list:
            
#             state_record_counter +=1
            
#             try:                
#                 querystring = {"iso":"USA", "region_province":state, "date":date_val}                
#                 response = requests.request("GET", url, headers=headers, params=querystring).json()

#                 new_state.append(response['data'][0]['region']['province'])
#                 lat_state.append(response['data'][0]['region']['lat'])
#                 long_state.append(response['data'][0]['region']['long'])
#                 total_cases_state.append(response['data'][0]['confirmed'])
#                 new_cases_state.append(response['data'][0]['confirmed_diff'])
#                 tot_deaths_state.append(response['data'][0]['deaths'])
#                 new_deaths_state.append(response['data'][0]['deaths_diff'])
#                 fatality_rate_state.append(response['data'][0]['fatality_rate'])
#                 date_state.append(response['data'][0]['date']) 
                
#                 if state_record_counter > 51:
#                     state_record_counter = 1
# #                     state_set_counter += 1
                    
#                 print(f'Processing record {state_record_counter} for set {state_set_counter} | {state} - {date_val}')
                
#             except:
#                 state_record_counter -= 1
#                 print(f'State error: {state} not found for {date_val}')
            
#     except:        
#         print(f'Date error: {state} is not found for {date_val}')
              
# print(f'---------------------------------Process Completed----------------------------------------')

Processing record 1 for set 1 | Alabama - 2020-10-11
Processing record 2 for set 1 | Alaska - 2020-10-11
Processing record 3 for set 1 | Arizona - 2020-10-11
Processing record 4 for set 1 | Arkansas - 2020-10-11
Processing record 5 for set 1 | California - 2020-10-11
Processing record 6 for set 1 | Colorado - 2020-10-11
Processing record 7 for set 1 | Connecticut - 2020-10-11
Processing record 8 for set 1 | Delaware - 2020-10-11
Processing record 9 for set 1 | District of Columbia - 2020-10-11
Processing record 10 for set 1 | Florida - 2020-10-11
Processing record 11 for set 1 | Georgia - 2020-10-11
State error: Hawai?i not found for 2020-10-11
Processing record 12 for set 1 | Idaho - 2020-10-11
Processing record 13 for set 1 | Illinois - 2020-10-11
Processing record 14 for set 1 | Indiana - 2020-10-11
Processing record 15 for set 1 | Iowa - 2020-10-11
Processing record 16 for set 1 | Kansas - 2020-10-11
Processing record 17 for set 1 | Kentucky - 2020-10-11
Processing record 18 for se

Processing record 45 for set 3 | Washington - 2020-10-13
Processing record 46 for set 3 | West Virginia - 2020-10-13
Processing record 47 for set 3 | Wisconsin - 2020-10-13
Processing record 48 for set 3 | Wyoming - 2020-10-13
---------------------------------Process Completed----------------------------------------


In [70]:
# Create dataframe based on state data
# covid_state_new_df = pd.DataFrame({
#     "State" : new_state,
#     "Total Cases" : total_cases_state,
#     "New Cases" : new_cases_state,
#     "Total Deaths" : tot_deaths_state,
#     "New Deaths" : new_deaths_state,
#     "Fatality Rate" : fatality_rate_state,
#     "Latitude" : lat_state,
#     "Longitude" : long_state,
#     "Date" : date_state    
# })    

# covid_state_new_df.to_csv('Resources/covid_state_df.csv', mode='a', index=False, header=False )
covid_state_new_df.head(2)

,State,Total Cases,New Cases,Total Deaths,New Deaths,Fatality Rate,Latitude,Longitude,Date
0,Alabama,165342,816,2664,0,0.0161,32.3182,-86.9023,2020-10-11
1,Alaska,9686,255,60,0,0.0062,61.3707,-152.4044,2020-10-11


In [49]:
state_data_path = 'Resources/covid_state_df.csv'
county_data_path = 'Resources/covid_county_df.csv'
covid_state_df = pd.read_csv(state_data_path)
covid_county_df = pd.read_csv(county_data_path)

In [68]:
# covid_state_df.drop(columns=['Unnamed: 0'], inplace = True)
df_ind = covid_state_df.set_index('State')
df_ind.loc['Indiana'].sort_values(by='Date', ascending=False)

,Total Cases,New Cases,Total Deaths,New Deaths,Fatality Rate,Latitude,Longitude,Date
State,,,,,,,,
Indiana,138104,1549,3822,27,0.0277,39.8494,-86.2583,2020-10-13
Indiana,136555,1574,3795,6,0.0278,39.8494,-86.2583,2020-10-12
Indiana,134981,1570,3789,7,0.0281,39.8494,-86.2583,2020-10-11
Indiana,133411,1918,3782,21,0.0283,39.8494,-86.2583,2020-10-10
Indiana,131493,1816,3761,19,0.0286,39.8494,-86.2583,2020-10-09
...,...,...,...,...,...,...,...,...
Indiana,645,168,17,3,0.0264,39.8494,-86.2583,2020-03-26
Indiana,477,109,14,2,0.0294,39.8494,-86.2583,2020-03-25
Indiana,368,98,12,5,0.0326,39.8494,-86.2583,2020-03-24


In [69]:
df_cal = covid_state_df.set_index('State')
df_cal.loc['California'].sort_values(by='Date', ascending=False)

,Total Cases,New Cases,Total Deaths,New Deaths,Fatality Rate,Latitude,Longitude,Date
State,,,,,,,,
California,861310,3397,16644,58,0.0193,36.1162,-119.6816,2020-10-13
California,857913,2723,16586,7,0.0193,36.1162,-119.6816,2020-10-12
California,855190,2780,16579,12,0.0194,36.1162,-119.6816,2020-10-11
California,852410,3334,16567,81,0.0194,36.1162,-119.6816,2020-10-10
California,849076,3806,16486,68,0.0194,36.1162,-119.6816,2020-10-09
...,...,...,...,...,...,...,...,...
California,3899,901,81,16,0.0208,36.1162,-119.6816,2020-03-26
California,2998,460,65,15,0.0217,36.1162,-119.6816,2020-03-25
California,2538,430,50,11,0.0197,36.1162,-119.6816,2020-03-24


In [54]:
covid_county_df.set_index('State')
mask1 = covid_county_df['County'] == 'Fulton'

covid_county_df=covid_county_df[mask1]
covid_county_df

,State,County,Total Cases,New Cases,Total Deaths,New Deaths,Date,Latitude,Longitude
81,Indiana,Fulton,0,0,0,0,3/22/2020,41.046991,-86.262527
230,Indiana,Fulton,0,0,0,0,3/23/2020,41.046991,-86.262527
379,Indiana,Fulton,0,0,0,0,3/24/2020,41.046991,-86.262527
528,Indiana,Fulton,0,0,0,0,3/25/2020,41.046991,-86.262527
677,Indiana,Fulton,0,0,0,0,3/26/2020,41.046991,-86.262527
...,...,...,...,...,...,...,...,...,...
30043,Indiana,Fulton,291,7,2,0,10/12/2020,41.046991,-86.262527
30192,Indiana,Fulton,294,3,2,0,10/13/2020,41.046991,-86.262527
30341,Indiana,Fulton,284,2,2,0,10/11/2020,41.046991,-86.262527
30490,Indiana,Fulton,291,7,2,0,10/12/2020,41.046991,-86.262527
